<a href="https://colab.research.google.com/github/hans321/CS4240/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True
zca_enable = True

In [2]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

ModuleNotFoundError: No module named 'wheel'

In [3]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

ERROR: Command errored out with exit status 1:
     command: 'c:\users\jan van tuijn\appdata\local\programs\python\python37-32\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\JANVAN~1\\AppData\\Local\\Temp\\pip-install-wfhpcnns\\torch\\setup.py'"'"'; __file__='"'"'C:\\Users\\JANVAN~1\\AppData\\Local\\Temp\\pip-install-wfhpcnns\\torch\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\JANVAN~1\AppData\Local\Temp\pip-record-kolv4qtj\install-record.txt' --single-version-externally-managed --compile --install-headers 'c:\users\jan van tuijn\appdata\local\programs\python\python37-32\Include\torch'
         cwd: C:\Users\JANVAN~1\AppData\Local\Temp\pip-install-wfhpcnns\torch\
    Complete output (23 lines):
    running install
    running build_deps
    Traceback (most recent call last):
      File "<string>

ModuleNotFoundError: No module named 'torchvision'

In [4]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

'wget' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'utils'

In [5]:
!wget https://github.com/hans321/CS4240/raw/master/Models.py
!wget https://github.com/hans321/CS4240/raw/master/zca.py
from Models import *
from zca import *

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'torch'

In [6]:
cuda = cuda and torch.cuda.is_available()

# Download trainset for normalization
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)

# Calculate mean and std for normalization
train_mean = trainset.data.mean(axis=(0, 1, 2)) / 255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0, 1, 2)) / 255    # [0.24703223  0.24348513  0.26158784]

# Apply tensor conversion and normalization on both training and test set
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

# Load training data
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)

# Load test data
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

# Load a single large training batch to performn ZCA on
zca_train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=50000, shuffle=False, **kwargs)

# Load a single large test batch to performn ZCA on
zca_test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=10000, shuffle=False, **kwargs)


if zca_enable:
    # Apply ZCA on training set
    dataiter = iter(zca_train_loader)
    images, labels = dataiter.next()
    train_zca = images.numpy().reshape(50000, 3*32*32)

    zca_train = ZCA()
    zca_train.fit(train_zca) 

    # Apply ZCA on test set
    dataiter = iter(zca_test_loader)
    images, labels = dataiter.next()
    test_zca = images.numpy().reshape(10000, 3*32*32)

    zca_test = ZCA()
    zca_test.fit(test_zca)         


NameError: name 'torch' is not defined

In [7]:
model = ALL_CNN_C(3)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
            torch.nn.init.xavier_uniform_(m.weight)
            torch.nn.init.zeros_(m.bias)
model.apply(weights_init)    

if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 250, 300], gamma=0.1)

NameError: name 'ALL_CNN_C' is not defined

In [8]:
def train(epoch):
    model.train()
    training_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if zca_enable:
          data = zca_train.transform(data.reshape(data.shape[0], 3*32*32))
          data = torch.tensor(data.reshape(data.shape[0], 3, 32, 32))        
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss
        loss.backward()
        optimizer.step()

        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))
    
    train_loss /= train_batch_size
    print('\Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          train_loss, correct, len(train_loader.dataset), 100. * correct / len(train_loader.dataset)))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if zca_enable: 
          data = zca_test.transform(data.reshape(data.shape[0], 3*32*32))
          data = torch.tensor(data.reshape(data.shape[0], 3, 32, 32))        
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= test_batch_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        if gsync_save:
            gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [9]:
for epoch in range(350):
    train(epoch)
    scheduler.step()
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

NameError: name 'model' is not defined

In [10]:
# if in Google Colab, download your model with this
from google.colab import files
files.download("best.pt")

ModuleNotFoundError: No module named 'google'